In [ ]:
from TendersWA.Graph.graph import GraphDB
import random
import numpy as np
import re
import pandas as pd


from ipywidgets import interact, widgets

import pandas as pd
from datetime import datetime

In [ ]:
# getting data 
date_data_path = r'../data/tender_postcode.xlsx'

path = r'../data/embedding_data/'
embedding_data = np.load(path+"sent_transformer_embeddings.npz")
embedding_data = [list(embedding_data[f]) for f in embedding_data.files]

tender_references_raw = ''
tender_titles_raw = ''

with open(path+'tender_references.txt', 'r') as file:
    for line in file:
        tender_references_raw += line

# getting a few encoding errors, unlikely to be an issue
with open(path+"tender_titles.txt",'r', errors = "ignore") as file:
    for line in file:
        tender_titles_raw += line
        
        
for item in ['"',"'",'[',']'," "]:
    tender_references_raw = tender_references_raw.replace(item, "")
tender_references = tender_references_raw.split(",")

tender_titles = []
for item in tender_titles_raw.split(","):
    tender_titles.append(re.sub(r'[\[\]/\\\'"]', '', item).lstrip())

# load clustering data    
topic_data = pd.read_csv("../data/clustering/sentence_bert_clusters.csv")
topic_data_list = list(topic_data['Cluster'])
cluster_data = pd.read_csv("../data/clustering/sentence_bert_cluster_topics.csv")
topics_list = list(cluster_data['Topics'])

coordinate_data = np.load(path+"tender_coordinates.npz")
coordinate_data = [list(coordinate_data[f]) for f in coordinate_data]

expire_dates = pd.read_excel(date_data_path)
expire_dates = list(expire_dates['Expiry Date'])

In [ ]:
# save to dictionary format
embedding_dictionary = {}
for i in range(len(tender_references)):
    embedding_dictionary[tender_references[i]] = {"embedding":embedding_data[i],"titles":tender_titles[i],"location":coordinate_data[i],"expire_date":expire_dates[i],"cluster":topic_data_list[i],'topics':topics_list[topic_data_list[i]]}

In [ ]:
graph_db = GraphDB(embedding_dictionary)

In [ ]:


start_date = datetime(2018, 1, 1)
end_date = datetime(2036, 1, 1)

dates = pd.date_range(start_date, end_date, freq='M')

options = [(date.strftime(' %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Tender Expiration date ',
    orientation='horizontal',
    layout={'width': '500px'}
)


In [ ]:

reference_input = widgets.Text(value='Please enter a value', description='Enter Reference :')
number_slider = widgets.IntSlider(value=2, min=1, max=10, step=1, description='Closest Tenders:')
distance_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Max Distance:')
tender_expire_slider = widgets.FloatSlider(value=100, min=1, max=300, step=1, description='Tender Expiry Month:')
tender_neighbour_slider = widgets.IntSlider(value=1, min=0, max=10, step=1, description='Level of Neighbours:')

topics_search = widgets.Checkbox(value=False,description='topics',disabled=False,indent=False)
topic_input = widgets.Text(value='Please enter a value', description='Enter topic number :')


interact(graph_db.query_and_display, reference=reference_input, num_nodes=number_slider, max_distance=distance_slider,expire_date=selection_range_slider,topic=topic_input,topic_box=topics_search,neighbours = tender_neighbour_slider)